In [15]:
import pandas as pd
from collections import Counter
from scipy.spatial.distance import cosine
import math
import re

In [29]:
Codalab = pd.read_csv('train_all_tasks.csv')
Exist2021 = pd.read_csv('EXIST2021_merged.csv')

In [30]:
data1 = Codalab.loc[Codalab['label_sexist'] == 'sexist']
data2 = Exist2021.loc[Exist2021['sexist'] == 1]
data2_nonsexist = Exist2021.loc[Exist2021['sexist'] == 0]

In [31]:
d1_text = []
for n1 in data1['text']:
  res1 = re.findall( r'\w+|[^\s\w]+', n1)
  d1_text.append(res1)

d2_text = []
for n2 in data2['text']:
  res2 = re.findall( r'\w+|[^\s\w]+', n2)
  d2_text.append(res2)

In [32]:
print(len(d1_text))
print(len(d2_text))

3398
2794


In [33]:
def cosine_dic(dic1, dic2, label_cat):
    numerator = 0
    dena = 0
    for key1,val1 in dic1.items():
        numerator += val1*dic2.get(key1,0.0)
        dena += val1*val1
    denb = 0
    for val2 in dic2.values():
        denb += val2*val2
    return numerator/math.sqrt(dena*denb), label_cat

In [40]:
def best_score(scores, sent):
  for score, label in sorted(zip(scores, sent), reverse=True)[:1]:
    return label

In [ ]:
cos_scores = []
cos_sent = []
Exist2021_label_categories = []
for n in d2_text:
  for n1, s1 in zip(d1_text, data1['label_category']):
    cs, lc = cosine_dic(Counter(n), Counter(n1), s1)
    cos_scores.append(cs)
    cos_sent.append(lc)

  Exist2021_label_categories.append(best_score(cos_scores, cos_sent))
  cos_scores = []
  cos_sent = []

In [ ]:
for x, y in zip(Exist2021_label_categories, d2_text):
  print(x, ' '.join(y))

In [ ]:
data2['label_category'] = Exist2021_label_categories
data2_nonsexist['label_category'] = 'none'

In [43]:
Exist2021_taskB = pd.concat([data2, data2_nonsexist], axis=0)
Exist2021_taskB = Exist2021_taskB.sample(frac=1)

In [46]:
Codalab['label_category'].value_counts()

none                                        10602
2. derogation                                1590
3. animosity                                 1165
4. prejudiced discussions                     333
1. threats, plans to harm and incitement      310
Name: label_category, dtype: int64

In [47]:
Exist2021_taskB['label_category'].value_counts()

none                                        2850
2. derogation                               1235
3. animosity                                 974
4. prejudiced discussions                    319
1. threats, plans to harm and incitement     266
Name: label_category, dtype: int64

In [48]:
Exist2021_taskB.to_csv('Exist2021_taskB.csv', index=False, header=True)